In [ ]:
import google.generativeai as genai

# Configure the API key (replace with your actual API key or environment variable)
genai.configure(api_key='YOUR_API_KEY')

# Load a generative model, for example, 'gemini-pro'
model = genai.GenerativeModel('gemini-1.5-flash')


/home/azar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import PyPDF2

# Open PDF file
with open("resume-1.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    resume = ""
    for page in reader.pages:
        resume += page.extract_text() + "\n"



In [ ]:
final_prompt = f"""
You are an information extraction system. 
Your task is to carefully analyze the following resume and return a JSON object 
with the following fields:

- name (string)
- surname (string)
- phone_number (string, or null if not found)
- email (string, or null if not found)
- location (string, e.g. city, country, or null if not found)
- certificates (list of strings, empty if none)
- experience (list of objects, each with:
    - job_title (string)
    - company (string)
    - start_date (string or null if not available)
    - end_date (string or "Present" if ongoing, or null if not available)
    - description (string summarizing responsibilities)
  )
- projects (list of objects, each with:
    - title (string)
    - description (string summarizing the project)
    - technologies (list of strings, empty if not available)
  )

Make sure the JSON is valid and well-formatted. 
If some fields are missing in the resume, return them as null or empty lists instead of skipping.

Resume text:
{resume}
"""


In [ ]:
final_prompt = f"""
Vous êtes un système d’extraction d’informations.  
Votre tâche est d’analyser attentivement le CV suivant et de retourner un objet JSON 
avec les champs suivants :

- nom (string)
- prénom (string)
- numéro_téléphone (string, ou null si non trouvé)
- email (string, ou null si non trouvé)
- lieu (string, ex. ville, pays, ou null si non trouvé)
- certificats (liste de chaînes de caractères, vide si aucun)
- expériences (liste d’objets, chacun avec :
    - poste (string)
    - entreprise (string)
    - date_début (string ou null si non disponible)
    - date_fin (string ou "Actuel" si en cours, ou null si non disponible)
    - description (string résumant les responsabilités)
  )

Assurez-vous que le JSON soit valide et bien formaté.  
Si certains champs sont absents du CV, retournez-les comme null ou listes vides au lieu de les ignorer.

Texte du CV :  
{resume}
"""


In [8]:
# Generate content
response = model.generate_content(final_prompt)
print(response.text)

```json
{
  "name": "Benmalek",
  "surname": "Zohir",
  "phone_number": "+213 559 503 713",
  "email": "azariomalek@gmail.com",
  "location": "Tlemcen, Algeria",
  "certificates": [
    "Oracle Cloud Infrastructure 2024 Generative AI Certified Professional",
    "AWS Cloud Practitioner Essentials",
    "Customizing and Evaluating LLMs using Amazon SageMaker JumpStart",
    "Building RAG Agents with LLMs (NVIDIA)",
    "Build Chat Applications with OpenAI and LangChain (365 Data Science)",
    "The Machine Learning Process A-Z (365 Data Science)",
    "MongoDB Python Developer Path",
    "EF SET Certificate (Level C1 Advanced)"
  ],
  "experience": [
    {
      "job_title": "Content Creator",
      "company": null,
      "start_date": "May 2024",
      "end_date": "Present",
      "description": "Created over 140 educational TikTok videos on Python topics such as RAG systems, chatbots, and data analysis. Conducted 2 workshops at Algeria2.0: –Data Analysis with Python –Advanced Python P

In [10]:
import json
import re
def clean_code_json(response: str) -> dict:
    """
    Cleans a JSON-like response that may include markdown code fences and invalid characters.
    Returns a valid Python dictionary.
    """
    # Step 1: Remove BOM or weird hidden characters
    response = response.strip().lstrip("\ufeff").strip()
    
    # Step 2: Extract JSON between first { and last }
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if not match:
        raise ValueError("No valid JSON object found in response")
    json_str = match.group(0)
    
    # Step 3: Parse JSON safely
    data = json.loads(json_str)
    return data

In [12]:
cleaned_data = clean_code_json(response.text)

In [13]:
cleaned_data

{'name': 'Benmalek',
 'surname': 'Zohir',
 'phone_number': '+213 559 503 713',
 'email': 'azariomalek@gmail.com',
 'location': 'Tlemcen, Algeria',
 'certificates': ['Oracle Cloud Infrastructure 2024 Generative AI Certified Professional',
  'AWS Cloud Practitioner Essentials',
  'Customizing and Evaluating LLMs using Amazon SageMaker JumpStart',
  'Building RAG Agents with LLMs (NVIDIA)',
  'Build Chat Applications with OpenAI and LangChain (365 Data Science)',
  'The Machine Learning Process A-Z (365 Data Science)',
  'MongoDB Python Developer Path',
  'EF SET Certificate (Level C1 Advanced)'],
 'experience': [{'job_title': 'Content Creator',
   'company': None,
   'start_date': 'May 2024',
   'end_date': 'Present',
   'description': 'Created over 140 educational TikTok videos on Python topics such as RAG systems, chatbots, and data analysis. Conducted 2 workshops at Algeria2.0: –Data Analysis with Python –Advanced Python Programming: Building a RAG System. Delivered a talk at Algeria2